In [23]:
from pathlib import Path
import pandas as pd
from prefect import flow, task
from prefect_gcp.cloud_storage import GcsBucket
from prefect_gcp import GcpCredentials
import io
import os
import requests
import calendar

In [24]:
@task(retries=3)
def extract_load_web_data(year:int , month:int,file_name) -> None:
    request_url =f"https://use-land-property-data.service.gov.uk/datasets/td/download/history/{calendar.month_name[month]}%20{year}/{file_name}.csv"
    r = requests.get(request_url)
    url_content=r.content
    csv_file = open(f'{file_name}.csv', 'wb')
    csv_file.write(url_content)
    csv_file.close()

/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/tasks.py:261: UserWarning: A task named 'extract_load_web_data' and defined at '/tmp/ipykernel_45968/1744432909.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


In [25]:
@task()
def load_csv_data (file_name: str) -> pd.DataFrame:
    df = pd.read_csv(f"{file_name}.csv", sep=',', header=4, index_col=False)
    return df

/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/tasks.py:261: UserWarning: A task named 'load_csv_data' and defined at '/tmp/ipykernel_45968/3339839138.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


In [26]:
@task()
def clean_local_data(df: pd.DataFrame) -> pd.DataFrame:
    """ We need to add a date column for each DataFrame """
    new_df = df[df["Account Customer"].str.contains("Total")==False]
    """ We are dropping the "Total" Column from the Dataframe """
    new_df = new_df.drop(["Total"], axis=1)
    new_df.columns = new_df.columns.str.replace("[( )]", "_")
    return new_df

/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/tasks.py:261: UserWarning: A task named 'clean_local_data' and defined at '/tmp/ipykernel_45968/3782913062.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


In [27]:
@task()
def transform_data(df: pd.DataFrame, year, month) -> None:
    df["date_added"] = f"01.{month:02}.{year}"
    return df

/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/tasks.py:261: UserWarning: A task named 'transform_data' and defined at '/tmp/ipykernel_45968/1433901114.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


In [28]:
@task()
def save_transformed_data(df: pd.DataFrame, file_name) -> None:
    df.to_parquet(f"{file_name}.parquet", compression="gzip", index=False)
    df = pd.read_parquet(f"{file_name}.parquet")
    return df

/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/tasks.py:261: UserWarning: A task named 'save_transformed_data' and defined at '/tmp/ipykernel_45968/4246081314.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


In [29]:
@task()
def write_gcs(file_name: str, path_gcp) -> None:
    """Upload local parquet FIle """
    gcp_cloud_storage_bucket_block = GcsBucket.load("gcp-project-dc")
    gcp_cloud_storage_bucket_block.upload_from_path(
        from_path = f"{file_name}.parquet",
        to_path=f"{path_gcp}/{file_name}"
    )

/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/tasks.py:261: UserWarning: A task named 'write_gcs' and defined at '/tmp/ipykernel_45968/1425389079.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


In [30]:
@task()
def write_bq(df: pd.DataFrame) -> None:
    """Write DataFrame to BigQuery"""
    gcp_credentials_block = GcpCredentials.load("gcp-project-2023")
    print(df.head())
    df.to_gbq(
        destination_table="project_dwh.land_and_property", # Dataset.TableName
        project_id="dc-project-2023",
        credentials=gcp_credentials_block.get_credentials_from_service_account(),
        chunksize=500.000,
        if_exists="append",
    )

/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/tasks.py:261: UserWarning: A task named 'write_bq' and defined at '/tmp/ipykernel_45968/671548081.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


In [31]:
@flow()
def etl_web_to_local_to_gcs(year: int, month: int) -> None:
    file_name = f"Number-and-types-of-applications-by-all-account-customers-{year}-{month:02}"
    extract_load_web_data(year, month, file_name)
    raw_df = load_csv_data(file_name)
    cleaned_df = clean_local_data(raw_df)
    transformed_df = transform_data (cleaned_df, year, month)
    final_df = save_transformed_data (transformed_df, file_name)
    write_gcs(file_name, "Land_and_Property")
    write_bq(final_df)
    

/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/flows.py:206: UserWarning: A flow named 'etl-web-to-local-to-gcs' and defined at '/tmp/ipykernel_45968/506873230.py:1' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


In [32]:
@flow()
def etl_parent_flow(years:list[int] , months:list[int]) -> None:
    for year in years:
        for month in months:
            etl_web_to_local_to_gcs(year, month)

/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/flows.py:206: UserWarning: A flow named 'etl-parent-flow' and defined at '/tmp/ipykernel_45968/1325823985.py:1' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


## Execution

In [33]:
if __name__ == '__main__':
    years=[2022]
    months=[2,3]
    etl_parent_flow(years, months) 

14:49:33.722 | INFO    | prefect.engine - Created flow run 'slick-grouse' for flow 'etl-parent-flow'

/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/tasks.py:261: UserWarning: A task named 'etl-web-to-local-to-gcs' and defined at '/tmp/ipykernel_45968/506873230.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


14:49:34.023 | INFO    | Flow run 'slick-grouse' - Created subflow run 'busy-dog' for flow 'etl-web-to-local-to-gcs'

14:49:34.159 | INFO    | Flow run 'busy-dog' - Created task run 'extract_load_web_data-0' for task 'extract_load_web_data'

14:49:34.161 | INFO    | Flow run 'busy-dog' - Executing 'extract_load_web_data-0' immediately...

...

14:49:49.523 | INFO    | Flow run 'macho-chowchow' - Executing 'write_bq-0' immediately...

       Account Customer  FR  DFL  TP  DLG  OS_W_  OS_NPW_  OS_P_  OS_NPP_  \
0     1 CLICK HOMES LTD   0    0   0    0      0        0      0        0   
1           15K LIMITED   0    0   0    0      0        0      0        0   
2           192.COM LTD   0    0   0    0      0        0      0        0   
3  1PLUS1 LOANS LIMITED   0    0   0    1      0        0      0        0   
4      1PM (UK) LIMITED   0    0   0    1      0        0      0        0   

   SIMS  OC1  OC2  date_added  
0     0    1    0  01.03.2022  
1     0    4    0  01.03.2022  
2     0  348    0  01.03.2022  
3     0   26    0  01.03.2022  
4     0   40    0  01.03.2022  


14:49:53.906 | INFO    | Task run 'write_bq-0' - Finished in state Completed()

14:49:53.973 | INFO    | Flow run 'macho-chowchow' - Finished in state Completed('All states completed.')

14:49:54.028 | INFO    | Flow run 'slick-grouse' - Finished in state Completed('All states completed.')